### Correlation functions using Lanczos basis

One can, using the necessary Lanczos basis, calculate the static and dynamical correlation functions at $T = 0$. Here, we assume  the knowledge of the non-degenerate ground state wavefunction $| \psi_0 \rangle$. For operator $\hat{A}$ and $\hat{B}$, the correlation function $C_{AB}(t,t')$ is defined as :

$$C_{AB}(t,t') = \langle \psi_0 \left| A(t) B(t') \right| \psi_0 \rangle, $$

If the time translation symmetry (ergodicity) is implied,

$$C_{AB}(t,t') =C_{AB}(t-t') = \langle \psi_0 \left| A(t-t') B(0) \right| \psi_0 \rangle, $$


For autocorrelations,
$$C_{AA}(t) = \langle \psi_0 \left| A(t) A^\dagger(0) \right| \psi_0 \rangle, $$

we can switch to the Fourier basis and obtain the Lehman representation by insertion of the resolution identity of many-body basis,

\begin{eqnarray}
\tilde{C}_{AA}(\omega) &=& \int_{-\infty}^{\infty} e^{i \omega t} C_{AA}(t) dt =\int_{-\infty}^{\infty} e^{-i \omega t}  \langle \psi_0 \left| A(t) A^\dagger(0) \right| \psi_0 \rangle, \\
&=& \sum_{m} \int_{-\infty}^{\infty} e^{i \omega t}  \langle \psi_0 \left| e^{i \hat{H} t} A(0) e^{-i \hat{H} t} \left| m \rangle \langle m\right|A^\dagger(0) \right| \psi_0 \rangle ,\\
&=& \sum_{m} \int_{-\infty}^{\infty} e^{i \omega t}  \langle \psi_0 \left| e^{i E_0 t} A(0) e^{-i E_m t} \left| m \rangle \langle m\right|A^\dagger(0) \right| \psi_0 \rangle,\\
&=&\sum_{m}  \left| \langle \psi_0 | A(0)  | m \rangle \right|^2 \int_{-\infty}^{\infty} e^{i(\omega-E_m+E_0) t }dt,\\
&=&\sum_{m}  \left| \langle \psi_0 | A(0)  | m \rangle \right|^2 \delta(\omega-E_m+E_0)
\end{eqnarray}

Yet, there is another way to obtain $\tilde{C}_{AA}(\omega)$ through the Green's functions,

$$\tilde{C}_{AA}(\omega) = \frac{-1}{\pi} Im G_A^<(\omega),\quad \text{where} \quad G_A^<(\omega) = \langle \psi_0 \left| A \frac{1}{\omega + i \eta -H + E_0} A^\dagger \right| \psi_0 \rangle$$

The representation for the lesser Gfnc comes from the Fourier multiplication of the time evolution of the operator $A(0)$.

Now, the task boils down to calculate $G_A^<(\omega)$. There is a continued fraction expansion of this function utilizing the Lanczos basis of 
$$|f_0 \rangle = A^\dagger | \psi_0 \rangle$$ 

Let us assume a Krylov basis generate by $|f_0 \rangle$ and $A^\dagger$: $\mathcal{H}_f = span\{ |f_0 \rangle, A^\dagger|f_0 \rangle,... ,(A^\dagger)^k |f_0 \rangle \} \equiv \{ |f_0 \rangle, |f_1 \rangle ,..., |f_{l} \rangle \}$

Note that $l \le k$ depending on the rank of $\mathcal{H}_f$. $U$ is a $l\times k$ matrix and $|f_m \rangle$'s has $k \times 1$ vector representation.

If one can express the resolvent in this basis, 

\begin{eqnarray}
\frac{1}{\omega + i \eta -H + E_0} &=& R = \sum_{nm=1}^l R_{nm} |f_n \rangle \langle f_m |
\end{eqnarray}

Inserting this expression back into the lesser Gfnc,
\begin{eqnarray}
G_A^<(\omega) &=& \sum_{nm=1}^l R_{nm} \langle f_0 \left|f_n \rangle \langle f_m \right| f_0 \rangle,\\
&=&R_{00} \langle v_p \left| U_{0p}^*   U_{oq}\right| v_q \rangle
\end{eqnarray}

I summary, we can write down the corresponding Gfnc as the continued fraction using the Krylov space matrix elements $a_n$ and $b_n$.
\begin{eqnarray}
G_A^<(\omega) &=& \frac{\langle f_0 \mid f_0 \rangle}{w + i\eta -  a_0 - \frac{b_0^2}{w+i\eta-a_1-\frac{b_1^2}{w+i\eta -b_2^2}}}
\end{eqnarray}

In [6]:
# in case needed, sometimes the figures can be drawn with a seperate window
# the following two command enforces it to draw 'inline'
%matplotlib
%matplotlib inline
# ignore the warnings: can be useful for too many imag. part warnings 
import warnings
warnings.filterwarnings('ignore')
from numpy import *
from scipy import *
import numpy as np
import scipy as sp
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.pylab import * # for plotting commands
import matplotlib.pyplot as plt
from numpy import (real,imag,matrix,linspace,array,complex)
from IPython.display import Image
from mpl_toolkits.mplot3d import axes3d

Using matplotlib backend: TkAgg


In [9]:
def LancFirat(H,Logsensitivity):
    aN = H.shape[0]
    v0 = np.random.rand(aN,)
    v0 = v0/norm(v0)
    threshold = 1e-2
    error = 1.
    count = 0
    maxIt = aN
    ee0,ee1,ee2,eer0,eer1 = [np.array(zeros((maxIt,),dtype='double')) for _ in range(5)]

    k = 3
    erLog = 1
    Trank = 3
    while erLog>Logsensitivity or abs(Trank-k)>4:
        MM = min(k,3)
        vjVec = np.array(zeros((aN,k),dtype='complex')) 
        aVec,bVec = [np.array(zeros((k,),dtype='complex')) for _ in range(2)]
        vjVec[:,0] = v0
        if k==3:
            vjVec[:,0] = v0
        else:
            vjVec[:,0:MM] = vv[:,0:MM]
        wjp = dot(H,vjVec[:,0])
        aVec[0]= dot(conj(wjp).T,vjVec[:,0])
        wj = wjp - aVec[0]*vjVec[:,0]

        for i2 in arange(1,k):
            bVec[i2] = norm(wj)
            if bVec[i2] != 0.:
                vj = wj/bVec[i2]
            else:
                vj = np.random.rand(aN,)
                print(i2)
            vSub = np.array(zeros((aN,),dtype='complex'))
            for i3 in arange(i2):
                vSub = vSub + dot(vjVec[:,i3],vj)*vjVec[:,i3]
            vj = vj-vSub 
            vj= vj/norm(vj)

            vjVec[:,i2] = vj
            wjp= dot(H,vj)
            aVec[i2] = dot(conj(wjp).T,vj)
            wj = wjp - aVec[i2]*vj - bVec[i2]*vjVec[:,i2-1]

        T = diag(aVec)+diag(bVec[1:],1)+diag(bVec[1:],-1)
        
        etI1,vtI1 = eig(T)
        ndee=argsort(etI1)
        vtI1 = vtI1[:,ndee[0:MM]]
        etI1 = etI1[ndee[0:MM]]

        error = norm(bVec)
        Trank = matrix_rank(vjVec)
        vv= dot(vjVec,vtI1)

        ee0[k-1] = etI1[0]
        ee1[k-1] = etI1[1]
        ee2[k-1] = etI1[2]
        eer0[k-1] = norm(etI1[0]*dot(vjVec,vtI1[:,0])-dot(H,dot(vjVec,vtI1[:,0])))
        eer1[k-1] = norm(etI1[1]*dot(vjVec,vtI1[:,1])-dot(H,dot(vjVec,vtI1[:,1])))
        
        #eer0[k-1] = abs(ee0[k-1]-(ee0[k-2]+ee0[k-3])/2.)
        #eer1[k-1] = abs(ee1[k-1]-(ee1[k-2]+ee1[k-3])/2.)
        erLog = log(eer0[k-1]+eer1[k-1])
        #print(erLog)
        eer = [eer0,eer1]
        k += 1
    return etI1,vv,k,ee0,ee1,ee2,eer0,eer1

In [10]:
from itertools import combinations
from itertools import permutations

def combs(N, X):
    idx = np.array(list(combinations(range(N), X)))
    n = len(idx)
    out = np.zeros((n,N), dtype=int)
    out[np.arange(n)[:,None],idx] = 1
    return out

In [11]:
def XXZfiratDirect(Jz,L,Nup):
    J = 1.
    UnVecs=combs(L,Nup)
    Hlen = len(UnVecs)
    diag0Vec = sum((UnVecs[:,0:L-1]-0.5)*(UnVecs[:,1:L]-0.5),1)
    interm = mod(UnVecs[:,0:L-1]+UnVecs[:,1:L],2)
    xL,yL = interm.shape
    bbOverlap = zeros((xL,yL,Hlen))
    for i1 in arange(Hlen):
        bbOverlap[:,:,i1] = interm[i1,:]*interm
    HIndicesSz0 = abs(np.dot(UnVecs,2**arange(L-1,-1,-1))-2**L+1)
    HIndArray = dot(ones((len(HIndicesSz0),1),dtype='double'),reshape(HIndicesSz0,[1,len(HIndicesSz0)]))
    hNeig = zeros((xL,xL),dtype='double')
    for i1 in arange(L):
        hNeig = hNeig + (abs(HIndArray.T-HIndArray)==2**i1).astype(int)
    H_direct = Jz*diag(diag0Vec,0) + 0.5*hNeig
    return H_direct

In [13]:
def CorrelatLanc(H,v0,E0,w,eta,Logsensitivity):
    H = H-eye(len(H))*E0
    v0 = v0/norm(v0)
    count = 0
    maxIt = len(H)
    
    k = 1
    erLog = 1
    Trank = 3
    Gfnc0 = 1e6
    while erLog>Logsensitivity: # or abs(Trank-k)>4:
        MM = min(k,3)
        vjVec = np.array(zeros((aN,k),dtype='complex')) 
        aVec,bVec = [np.array(zeros((k,),dtype='complex')) for _ in range(2)]
        vjVec[:,0] = v0

        wjp = dot(H,vjVec[:,0])
        aVec[0]= dot(conj(wjp).T,vjVec[:,0])
        wj = wjp - aVec[0]*vjVec[:,0]
        Gfnc0 = (w-1j*eta+E0-a0)**(-1)
        for i2 in arange(1,k):
            bVec[i2] = norm(wj)
            if bVec[i2] != 0.:
                vj = wj/bVec[i2]
            else:
                vj = np.random.rand(aN,)
                print(i2)
            vSub = np.array(zeros((aN,),dtype='complex'))
            for i3 in arange(i2):
                vSub = vSub + dot(vjVec[:,i3],vj)*vjVec[:,i3]
            vj = vj-vSub 
            vj= vj/norm(vj)

            vjVec[:,i2] = vj
            wjp= dot(H,vj)
            aVec[i2] = dot(conj(wjp).T,vj)
            wj = wjp - aVec[i2]*vj - bVec[i2]*vjVec[:,i2-1]
            aaInter = 0.
            for i3 in arange(k-1,-1,-1):
                aaInter = (w-1j*eta+E0-aVec[i3-1]-bVec[i3]*aaInter)**(-1)
            erLog = log(abs(aaInter-Gfnc0))
            Gfnc0 = aaInter  
        k += 1
    return Gfnc0

In [12]:
L = 10
Jz = 1.
H_Heis = XXZfiratDirect(Jz,L,int(L/2))
etI,vtI,k,ee0,ee1,ee2,eer0,eer1 = LancFirat(H_Heis,-20)

In [ ]:
We now obtained the the ground state energy and the wavefunction. The next task is to choose a correlation to calculate. We choose,

In [5]:
GfncLesser = CorrelatLanc(H,v0,E0,w,eta,Logsensitivity)

array([3, 2, 1, 0])